In [1]:
import pandas as pd

In [4]:
df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\classification_new.xlsx')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7526 entries, 0 to 7525
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SterilizationMethodId  7526 non-null   int64 
 1   MachineTypeId          7526 non-null   int64 
 2   ItemTypeId             7526 non-null   int64 
 3   SpecialtyId            7526 non-null   int64 
 4   CategoryName           7526 non-null   object
dtypes: int64(4), object(1)
memory usage: 294.1+ KB


In [11]:
# Group by CategoryName and count occurrences
category_counts = df['CategoryName'].value_counts()
category_counts

CategoryName
Instrument        5501
Power System       803
Rigid Scope        799
Flexible Scope     234
Video               93
Ultrasound          66
Ophthalmic          20
Endocam             10
Name: count, dtype: int64

In [16]:
# Filter the DataFrame based on the condition where CategoryName is "instrument"
instrument_df = df[df['CategoryName'] == 'Instrument']

# Select the bottom 2500 records from instrument_df
bottom_2500_instrument_df = instrument_df.tail(2500)

# Remove the bottom 2500 records from the original DataFrame
filtered_df = df.drop(bottom_2500_instrument_df.index)

In [48]:
# Use to_excel() method to save DataFrame to Excel
file_path = r'C:\Users\pawvaibh\Downloads\Removed_Instruments.xlsx'
bottom_2500_instrument_df.to_excel(file_path, index=False)

In [22]:
# Group by CategoryName and count occurrences
category_counts = filtered_df['CategoryName'].value_counts()
category_counts

CategoryName
Instrument        3001
Power System       803
Rigid Scope        799
Flexible Scope     234
Video               93
Ultrasound          66
Ophthalmic          20
Endocam             10
Name: count, dtype: int64

In [23]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5026 entries, 0 to 7314
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   SterilizationMethodId  5026 non-null   int64 
 1   MachineTypeId          5026 non-null   int64 
 2   ItemTypeId             5026 non-null   int64 
 3   SpecialtyId            5026 non-null   int64 
 4   CategoryName           5026 non-null   object
dtypes: int64(4), object(1)
memory usage: 235.6+ KB


In [24]:
df = filtered_df

In [80]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

# Combine categories except "Instrument" into one category called "Others"
def combine_categories(df):
    df.loc[~df['CategoryName'].isin(['Instrument']), 'CategoryName'] = 'Others'
    return df

def segregate(df):
    X = df.drop(columns=['CategoryName'])
    y = df['CategoryName']
    return X, y
    
# Train and evaluate RandomForestClassifier
def train_and_evaluate_model(X, y):
    model = RandomForestClassifier(random_state=42)
    
    model.fit(X, y)
    
    # Predict on the training and testing data
    predictions = model.predict(X)
    
    return model

# Preprocess the data
df1 = df.copy()
df2 = df.copy()
df3 = df.copy()
df4 = df.copy()

# Model 1: Classify Instrument vs. Others
df1 = combine_categories(df1)

X1, y1 = segregate(df1)

print("Model 1:")

model_first = train_and_evaluate_model(X1, y1)

test_df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\Instruments.xlsx')

# Iterate over each row in the test data
for index, row in test_df.iterrows():
    # Extract the input features from the row
    new_input = row[['SterilizationMethodId', 'MachineTypeId', 'ItemTypeId', 'SpecialtyId']].values.reshape(1, -1)
    # Make predictions using the trained models
    predicted_category = model_first.predict(new_input)

    if predicted_category == 'Others':
        # Model 2: Classify Power System, Rigid Scope vs. Others
        df2 = df2[~df2['CategoryName'].isin(['Instrument'])]
        df2.loc[~df2['CategoryName'].isin(['Power System', 'Rigid Scope']), 'CategoryName'] = 'Others'

        X2, y2 = segregate(df2)

        print("\nModel 2:")
        model_sec = train_and_evaluate_model(X2, y2)
    
        predicted_category = model_sec.predict(new_input)
        print("Predicted Category from Model 2:", predicted_category)
    
        if predicted_category == 'Others':
            # Model 3: Classify Flexible Scope, Video Vs. Others
            df3 = df3[~df3['CategoryName'].isin(['Instrument', 'Rigid Scope', 'Power System'])]
            df3.loc[~df3['CategoryName'].isin(['Flexible Scope', 'Video']), 'CategoryName'] = 'Others'

            X3, y3 = segregate(df3)
            
            print("\nModel 3:")
            model_third = train_and_evaluate_model(X3, y3)
        
            predicted_category = model_third.predict(new_input)
            print("Final Predicted Category from Model 3:", predicted_category)

            if predicted_category == 'Others':
                # Model 4: Classify 'Ultrasound', 'Opthamlmic', 'Endocam'
                df4 = df4[df4['CategoryName'].isin(['Ultrasound', 'Opthamlmic', 'Endocam'])]

                X4, y4 = segregate(df4)
               
                print("\nModel 4:")
                model_last = train_and_evaluate_model(X4, y4)
        
                predicted_category = model_last.predict(new_input)
                print("Final Predicted Category from Model 4:", predicted_category)
                # Update the 'Predicted_Values' column with the predicted category
                test_df.at[index, 'Predicted_Values'] = predicted_category

            else:
                print("Final Predicted Category from Model 3:", predicted_category)
                # Update the 'Predicted_Values' column with the predicted category
                test_df.at[index, 'Predicted_Values'] = predicted_category
        else:
            print("Final Predicted Category from Model 2:", predicted_category)
            # Update the 'Predicted_Values' column with the predicted category
            test_df.at[index, 'Predicted_Values'] = predicted_category
    else:
        print("Final Predicted Category from Model 1:", predicted_category)
        # Update the 'Predicted_Values' column with the predicted category
        test_df.at[index, 'Predicted_Values'] = predicted_category

# Save the updated DataFrame back to the Excel file
test_df.to_excel(r'C:\Users\pawvaibh\Downloads\Predicted_Output.xlsx', index=False)

Model 1:


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\Temp\ipykernel_21920\2326355904.py:99: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Power System' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_df.at[index, 'Predicted_Values'] = predicted_category



Model 2:
Predicted Category from Model 2: ['Power System']
Final Predicted Category from Model 2: ['Power System']


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(



Model 2:
Predicted Category from Model 2: ['Power System']
Final Predicted Category from Model 2: ['Power System']

Model 2:


...

C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with featur

Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']

Model 2:


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with featur

Predicted Category from Model 2: ['Power System']
Final Predicted Category from Model 2: ['Power System']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']

Model 2:
Predicted Category from Model 2: ['Power System']
Final Predicted Category from Model 2: ['Power System']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Category from Model 1: ['Instrument']
Final Predicted Catego

C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with featur

In [81]:

# Read the predicted values and true values from the Excel file
val_df = pd.read_excel(r'C:\Users\pawvaibh\Downloads\Predicted_Output.xlsx')

# Extract true and predicted values
true_values = val_df['True_Values']
predicted_values = val_df['Predicted_Values']

# Get unique classes from both true and predicted values
unique_classes_true = true_values.unique()
unique_classes_predicted = predicted_values.unique()

# Calculate evaluation metrics
accuracy = accuracy_score(true_values, predicted_values)
precision = precision_score(true_values, predicted_values, average='weighted')
recall = recall_score(true_values, predicted_values, average='weighted')
f1 = f1_score(true_values, predicted_values, average='weighted')
conf_matrix = confusion_matrix(true_values, predicted_values)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Print the unique classes
print("Unique classes in true labels:", unique_classes_true)
print("Unique classes in predicted labels:" , unique_classes_predicted)

print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.6770428015564203
Precision: 0.7687753161045104
Recall: 0.6770428015564203
F1 Score: 0.7064869144531091
Unique classes in true labels: ['Power System' 'Rigid Scope' 'Flexible Scope' 'Video' 'Ultrasound'
 'Ophthalmic' 'Endocam' 'Instrument']
Unique classes in predicted labels: ['Power System' 'Instrument' 'Rigid Scope' 'Flexible Scope' 'Ultrasound'
 'Video' 'Endocam']
Confusion Matrix:
[[  1   0   0   0   0   2   0   0]
 [  0   6   7   0  11  15   0   1]
 [  0   2 612   0  65  88   6   4]
 [  0   0   2   0   2   0   0   0]
 [  0   2  38   0  41  54   3   2]
 [  0   0  12   0   1  33   0   0]
 [  0   0   1   0   0   5   2   0]
 [  0   1   0   0   0   7   1   1]]


C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [82]:
# Generate classification report
report = classification_report(true_values, predicted_values)

# Print classification report
print(report)

                precision    recall  f1-score   support

       Endocam       1.00      0.33      0.50         3
Flexible Scope       0.55      0.15      0.24        40
    Instrument       0.91      0.79      0.84       777
    Ophthalmic       0.00      0.00      0.00         4
  Power System       0.34      0.29      0.32       140
   Rigid Scope       0.16      0.72      0.26        46
    Ultrasound       0.17      0.25      0.20         8
         Video       0.12      0.10      0.11        10

      accuracy                           0.68      1028
     macro avg       0.41      0.33      0.31      1028
  weighted avg       0.77      0.68      0.71      1028



C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pawvaibh\AppData\Local\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
